In [1]:
import os
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser

from langchain.llms.vllm import VLLMOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from pdf_to_vectorstore import get_pdf_text
from langchain.text_splitter import CharacterTextSplitter

In [2]:
def initiate_mixtral():

    load_dotenv()
    mixtral_api_key = os.getenv("MIXTRAL_API_KEY")
    mixtral_api_base = os.getenv("MIXTRAL_API_BASE")

    llm = VLLMOpenAI(
        openai_api_key=mixtral_api_key,
        openai_api_base=mixtral_api_base,
        model_name="mistralai/Mixtral-8X7B-Instruct-v0.1",
        temperature=0.0,
        max_tokens=7000,
    )
    return llm

In [3]:
mixtralllm = initiate_mixtral()

In [4]:

from PyPDF2 import PdfReader


def get_pdf_text(pdf_docs):  # Objectif : Lire le texte de plusieurs fichiers PDF.
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)  # on lit chaque pdf
        for page in pdf_reader.pages:
            text += (
                page.extract_text()
            )  # On extrait le texte des pdf lu et on l'ajoute à text
    return text


In [5]:
from langchain.document_loaders import PyPDFLoader

INPUT_FILE="./pdf_doc/Compromis.pdf"
loader = PyPDFLoader(INPUT_FILE)
docs = loader.load()

In [6]:
doc_part1= docs[0:18]
doc_part2= docs[18:]


In [7]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate




prompt_template = """Voici mon texte:
"{text}"
Transforme cela en un résumé final qui sera organisé sous les sections suivantes:
"Vendeur et Acquéreur" 
"Bien Immobilier" 
"Hypothèque et servitudes"
"Dossier du Diagnostic Technique (DDT)"
"Montant final et Modalités de Paiement"
"Promesse de Vente et Signature de Acte de Vente"
"Montant des Indemnité et Immobilisation et Conditions Suspensives"

Tu rempliras chaque section avec les informations adéquates en organisant ton compte-rendu en bullet points. Tu écriras chaque titre de section.
Si contenu pas mentionné, NE invente PAS de réponse.
"""




prompt = PromptTemplate.from_template(prompt_template)
# Define LLM chain
llm = mixtralllm
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")



print(stuff_chain.invoke(doc_part1)["output_text"])

c:\Users\ClementTEULIER\AppData\Local\pypoetry\Cache\virtualenvs\ov-project-JSYaqUAi-py3.12\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [19]:
print(stuff_chain.invoke(doc_part2)["output_text"])


Coordonnées du Vendeur et de l'Acquéreur
- Coordonnées du Vendeur
- Coordonnées de l'Acquéreur

Bien Immobilier
- Adresse du bien
- Type de bien
- Surface habitable
- Nombre de pièces
- Prix de vente

Hypothèque et servitudes
- Absence d'hypothèque
- Absence de servitudes

Dossier du Diagnostic Technique (DDT)
- Amiante: présence dans les matériaux et produits, évaluation périodique recommandée
- Electricité: anomalies constatées, état de l'installation intérieure d'électricité établi le 18 novembre 2021
- Gaz: absence d'installation intérieure de gaz
- Performance énergétique: consommation énergétique finale de 121 kwh/m²/an, émissions de gaz à effet de serre de 9 kgco2/m²/an
- Termites: absence de termites
- Mérule: absence de mérule
- Risques et pollution: absence de risques naturels, risque sismique de niveau 1, commune située en zone à potentiel radon significatif
- Assainissement: raccordement au réseau public d'assainissement collectif

Montant et Modalités de Paiement
- Prix d

In [ ]:
resume1= stuff_chain.invoke(doc_part1)["output_text"]
resume2= stuff_chain.invoke(doc_part2)["output_text"]